In [1]:
from shareUtillities import downloadSymbolHistory
import pandas as pd
from pandasql import sqldf

In [2]:
def saveDataframe(df, filename):
    df.to_pickle("./"+filename)

def loadDataframe(filename):
    df = pd.read_pickle("./"+filename)
    return df

In [3]:
def register(symbols,volumes, prices,dates,actions, currencies):
    pf = pd.DataFrame(
    {'symbol': symbols,
     'volume': volumes,
     'price' : prices,
     'currency':currencies,
     'date'  :dates,
     'action':actions
    })
    pf['dt']=pf['date'].dt.date
    df = loadDataframe('portfolio.pkl')
    df2=df.append(pf, sort=False)
    saveDataframe(df2, 'portfolio.pkl')

def truncatePortfolio():
    df2=loadDataframe('portfolio.pkl')
    # rop al rows
    df2.drop(df2.index, inplace=True)
    saveDataframe(df2, 'portfolio.pkl')

def getTradingDays(symbol):
    data, errorCode = sh.downloadSymbolHistory(symbol,'2999-12-31', False)
    return data.index.tolist()

def getFirstAquisitonDate(portfolio,symbol):
    firstAquisitionDate=sqldf("select min(dt) from portfolio where symbol = '"+symbol+"'")

truncatePortfolio()
register(['DANSKE.CPH'],[25],[154.10],[pd.Timestamp('2014-05-05 09:00')],['Buy'],['DKK'])
#register(['SPNO.CPH'],[106],[56.50],[pd.Timestamp('2014-11-20 09:00')],['Buy'],['DKK'])
#register(['DANSKE.CPH'],[24],[167.90],[pd.Timestamp('2015-02-12 09:00')],['Buy'],['DKK'])
#register(['ISS.CPH'],[24],[205.40],[pd.Timestamp('2015-03-12 09:00')],['Buy'],['DKK'])
#register(['DANSKE.CPH'],[25],[398],[pd.Timestamp('2014-05-05 09:00')],['Buy'],['DKK'])
#register(['DANSKE.CPH'],[30],[190.30],[pd.Timestamp('2015-07-10 09:00')],['Buy'],['DKK'])

portfolio=loadDataframe('portfolio.pkl')
dt=getFirstAquisitonDate(portfolio,'DANSKE.CPH')

In [4]:
portfolio

,action,currency,date,dt,price,symbol,volume
0,Buy,DKK,2014-05-05 09:00:00,2014-05-05,154.1,DANSKE.CPH,25.0


In [5]:
isinDict={}
isinDict['DK0010274414']='DANSKE.CPH'
isinDict['DK0060036564']='SPNO.CPH'
isinDict['DK0060542181']='ISS.CPH'
isinDict['DK0060534915']='NOVO-B.CPH'
isinDict['DK0060252690']='PNDORA.CPH'
#isinDict['US02079K3059']='GOOGL'
isinDict['US0378331005']='AAPL'
isinDict['US9311421039']='WMT'

import pandas as pd 
file='CDepotberegningBevaegelsereksportcsv.csv'
headers = ['Papir', 'ISIN', 'Dato for bevægelse', 'Dato for afvikling','Hændelse','Nota','Bevægelse','Beholdning','Afregningsbeløb','Kurs','Valuta','Valutakurs','Ukendt']
dtypes  = {'ISIN': 'str','Hændelse':'str','Valuta':'str'}
remove_thousand_sep_and_cast_to_float = lambda x: pd.np.float(x.replace(".", "").replace(',','.')) 
toString = lambda x: pd.np.str(x) 


parse_dates = ['Dato for bevægelse', 'Dato for afvikling']

d=pd.read_csv(file, sep=';', header=None, skiprows=1, names=headers, dtype=dtypes, 
              parse_dates=parse_dates, decimal=',', 
              converters={"Bevægelse": remove_thousand_sep_and_cast_to_float,
                          "Beholdning": remove_thousand_sep_and_cast_to_float,
                          "Afregningsbeløb": remove_thousand_sep_and_cast_to_float,
                          "Papir":toString})

def getCurrency(row):
    if 'CPH' in row['symbol']:
        return 'DKK'
    else:
        return 'USD'
    
truncatePortfolio()

for index, row in d.iterrows():
    symbol=isinDict.get(row['ISIN'],None)
    if symbol!=None:
        register([symbol],[row['Bevægelse']],[row['Kurs']],[row['Dato for afvikling']],['Buy'],['DKK'])
portfolio=loadDataframe('portfolio.pkl')
portfolio['currency'] = portfolio.apply(getCurrency, axis=1)

portfolio


,action,currency,date,dt,price,symbol,volume
0,Buy,DKK,2014-05-05,2014-05-05,154.1,DANSKE.CPH,25.0
0,Buy,DKK,2014-11-20,2014-11-20,56.5,SPNO.CPH,106.0
0,Buy,DKK,2015-12-02,2015-12-02,167.9,DANSKE.CPH,24.0
0,Buy,DKK,2015-03-16,2015-03-16,205.4,ISS.CPH,24.0
0,Buy,DKK,2015-04-29,2015-04-29,398.0,NOVO-B.CPH,12.0
0,Buy,DKK,2015-10-07,2015-10-07,190.3,DANSKE.CPH,30.0
0,Buy,DKK,2016-02-18,2016-02-18,53.5,SPNO.CPH,93.0
0,Buy,DKK,2016-03-30,2016-03-30,846.0,PNDORA.CPH,2.0
0,Buy,DKK,2016-05-26,2016-05-26,992.5,PNDORA.CPH,10.0
0,Buy,DKK,2016-07-09,2016-07-09,66.0,SPNO.CPH,113.0


In [38]:
from datetime import timedelta
for i, currencyCode in enumerate(portfolio.query('currency!="DKK"')['currency'].unique()):
    if i==0:
        currencyData,errorCoe = downloadSymbolHistory(symbol=currencyCode, maxDate='2999-12-31', 
                                                      download=True, type='currency', to_symbol='DKK')
    else:
        currencyData_,errorCoe = downloadSymbolHistory(symbol=currencyCode, maxDate='2999-12-31', 
                                                      download=True, type='currency', to_symbol='DKK')
        currencyData.append(currencyData_)
        
def getExchangeRate(currencyData, date, currencyCode, lookback=5):
    '''
    Will look up currency exchange rate on any give date
    If date does not exist, it will look back days to fin lates.
    Default lookback is 5 days
    '''
    for d in range(0,lookback):
        dateToTry=date-timedelta(days=d)
        try:
            return currencyData[currencyData.dt==dateToTry]['Close'].apply(float)[0]
        except:
            pass
    raise ValueError("No valid exchange rate found for date "+str(date))
            
import datetime        
d=datetime.datetime.strptime('2017-03-06', '%Y-%m-%d')
#currencyData[currencyData.dt==d]['Close'].apply(float)[0]

getExchangeRate(currencyData,d,'USD')
#currencyData.dtypes

For USD to DKK
Note: Downloading time series data from https://www.alphavantage.co/query


7.0239

In [14]:
portfolio.dtypes

action              object
currency            object
date        datetime64[ns]
dt                  object
price              float64
symbol              object
volume             float64
dtype: object

In [39]:

# Downloading closing prices
from pandasql import sqldf
def generateSymbolHistory(portfolio):
    global currencyData
    for i, symbol in enumerate(portfolio.symbol.unique()):
        print('**'+symbol)
        # Get the actions for symbols
        # Reformat the date column to join
        symbolActions = portfolio.query('symbol=="'+symbol+'"')[['dt','symbol','volume','currency']]
        currency=symbolActions.currency.unique()[0]
        print(currency)
        symbolActions['dt']=pd.to_datetime(symbolActions.dt,format='%Y-%m-%d')
        
        # Download pricedata and reforma date column for join
        priceData, errorCode=downloadSymbolHistory(symbol, maxDate='2999-12-31', download=True)  
        priceData['dt']=pd.to_datetime(priceData.index,format='%Y-%m-%d')
        
        # Join, sort an calculate acumulating sum
        priceData=priceData[['dt','Close']]
        out = priceData.merge(symbolActions, on=['dt', 'dt'], how='outer').fillna(0)#.set_index('dt')
        #print(out)
        out2 = out.sort_values(by='dt')
        out2[symbol+'_volume'] = out2['volume'].cumsum(axis = 0)
        out2[symbol+'_price'] = pd.to_numeric(out2['Close'], errors='coerce')
        #print(out2)
        if currency!='DKK':
            exchangedPrices=[]
            for iterNumber,row in out2.iterrows():
                exchangedPrices.append(row[symbol+'_price']*getExchangeRate(currencyData,row['dt'],currency))
        
        out2[symbol+'_value']=out2[symbol+'_volume']*out2[symbol+'_price']#pd.to_numeric(out2['Close'],errors='coerce')
        if i==0:
            result=out2
        else:
            result=out2.merge(result, on=['dt', 'dt'], how='outer').fillna(0)
    keepColumns = [c for c in result.columns if not '_x' in c.lower() and not '_y' in c.lower() 
                   and not 'symbol' in c.lower()
                   and not 'volume' == c.lower()]
    result=result[keepColumns]
    result['totalValue']=result.filter(like='_value').sum(axis=1)
    return result.sort_values(by='dt')
historicValues=generateSymbolHistory(portfolio)
historicValues.tail(20)
historicValues=historicValues.query('dt>="'+str(historicValues.query('totalValue>10')['dt'].min())+'"').query('totalValue>500')


**DANSKE.CPH
DKK
For DANSKE.CPH
Note: Downloading time series data from https://www.alphavantage.co/query
**SPNO.CPH
DKK
For SPNO.CPH
Note: Downloading time series data from https://www.alphavantage.co/query
**ISS.CPH
DKK
For ISS.CPH
Note: Downloading time series data from https://www.alphavantage.co/query
**NOVO-B.CPH
DKK
For NOVO-B.CPH
Note: Downloading time series data from https://www.alphavantage.co/query
**PNDORA.CPH
DKK
For PNDORA.CPH
Note: Downloading time series data from https://www.alphavantage.co/query
**WMT
USD
For WMT
Note: Downloading time series data from https://www.alphavantage.co/query


ValueError: No valid exchange rate found for date 1998-01-02 00:00:00

In [30]:
priceData, errorCode=downloadSymbolHistory('DANSKE.CPH', maxDate='2999-12-31', download=True)  
priceData

For DANSKE.CPH
Note: Downloading time series data from https://www.alphavantage.co/query


,Close,Relative,dt
Date,,,
2000-11-08,121.2230,0.000000,2000-11-08
2000-11-09,119.3730,-0.015498,2000-11-09
2000-11-10,123.0740,0.030071,2000-11-10
2000-11-13,121.2230,-0.015269,2000-11-13
2000-11-14,121.2230,0.000000,2000-11-14
2000-11-15,117.5220,-0.031492,2000-11-15
2000-11-16,124.9250,0.059260,2000-11-16
2000-11-17,129.5520,0.035715,2000-11-17
2000-11-20,127.7010,-0.014495,2000-11-20


In [26]:
historicValues.query('totalValue<3000 & totalValue>0')

,dt,Close,AAPL_volume,AAPL_price,AAPL_value,WMT_volume,WMT_price,WMT_value,PNDORA.CPH_volume,PNDORA.CPH_price,...,ISS.CPH_volume,ISS.CPH_price,ISS.CPH_value,SPNO.CPH_volume,SPNO.CPH_price,SPNO.CPH_value,DANSKE.CPH_volume,DANSKE.CPH_price,DANSKE.CPH_value,totalValue
4851,2017-04-13,141.0500,6.0,141.05,846.30,11.0,73.15,804.65,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1650.95
4852,2017-04-17,141.8300,6.0,141.83,850.98,11.0,73.49,808.39,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1659.37
4871,2017-05-12,156.1000,6.0,156.10,936.60,11.0,75.71,832.81,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1769.41
4880,2017-05-25,153.8700,6.0,153.87,923.22,11.0,78.31,861.41,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1784.63
4881,2017-05-26,153.6100,6.0,153.61,921.66,11.0,78.13,859.43,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1781.09
4886,2017-06-05,153.9300,6.0,153.93,923.58,11.0,80.26,882.86,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1806.44
5028,2017-12-26,170.5700,6.0,170.57,1023.42,11.0,99.16,1090.76,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2114.18
5092,2018-03-29,167.7800,6.0,167.78,1006.68,11.0,88.97,978.67,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1985.35
5093,2018-04-02,166.6800,6.0,166.68,1000.08,11.0,85.55,941.05,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1941.13
5112,2018-04-27,162.3200,6.0,162.32,973.92,11.0,87.29,960.19,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1934.11


In [9]:
portfolio.symbol

0    DANSKE.CPH
0      SPNO.CPH
0    DANSKE.CPH
0       ISS.CPH
0    NOVO-B.CPH
0    DANSKE.CPH
0      SPNO.CPH
0    PNDORA.CPH
0    PNDORA.CPH
0      SPNO.CPH
0    DANSKE.CPH
0       ISS.CPH
0           WMT
0          AAPL
0    DANSKE.CPH
0    PNDORA.CPH
0    NOVO-B.CPH
Name: symbol, dtype: object

In [10]:
import matplotlib.pyplot as plt
plot=True
if plot==True:
            f=plt.figure(figsize=(20,10))
            f.suptitle('symbol', fontsize=16)
            ax = f.add_subplot(111)
            ax.set_title('title ')
            ax.plot(historicValues.dt, historicValues['totalValue'], label="SMA short", color="blue")
            #ax.plot(historicValues.dt, historicValues.AAPL_price, label="SMA short", color="red")
            #ax.plot(daysDate, result.sma_long,  label="SMA long", color="blue")
            #ax.plot(daysDate, pd.to_numeric(result.Close, errors='coerce'), color="black")
            #ax.plot(daysDate, result.Close, color="black")
            ax.set_ylabel('Closing price', color='black')
            ax.legend(loc='lower center')
            
            #ax2 = ax.twinx()
            #ax2.plot(daysDate, result.rsi, 'r-', color='grey')
            #ax2.set_ylabel('Relative Strength Index (RSI)', color='grey')
            
            #horizontal30 = np.array([30 for i in range(len(daysDate))])  
            #ax2.plot(daysDate, horizontal30, 'r-', color='grey')
            #horizontal70 = np.array([70 for i in range(len(daysDate))])  
            #ax2.plot(daysDate, horizontal70, 'r-', color='grey')
            
            plt.show()

<Figure size 2000x1000 with 1 Axes>

In [11]:
portfolio
#truncatePortfolio()
#register(['DANSKE.CPH'],[25],[154.10],[pd.Timestamp('2014-05-05 09:00')],['Buy'])

,action,currency,date,dt,price,symbol,volume
0,Buy,DKK,2014-05-05,2014-05-05,154.1,DANSKE.CPH,25.0
0,Buy,DKK,2014-11-20,2014-11-20,56.5,SPNO.CPH,106.0
0,Buy,DKK,2015-12-02,2015-12-02,167.9,DANSKE.CPH,24.0
0,Buy,DKK,2015-03-16,2015-03-16,205.4,ISS.CPH,24.0
0,Buy,DKK,2015-04-29,2015-04-29,398.0,NOVO-B.CPH,12.0
0,Buy,DKK,2015-10-07,2015-10-07,190.3,DANSKE.CPH,30.0
0,Buy,DKK,2016-02-18,2016-02-18,53.5,SPNO.CPH,93.0
0,Buy,DKK,2016-03-30,2016-03-30,846.0,PNDORA.CPH,2.0
0,Buy,DKK,2016-05-26,2016-05-26,992.5,PNDORA.CPH,10.0
0,Buy,DKK,2016-07-09,2016-07-09,66.0,SPNO.CPH,113.0
